In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models.unet import UNet
from models.HybridGNet2IGSC import Hybrid 

import os 
import numpy as np
from torchvision import transforms
import torch

from utils.utils import scipy_to_torch_sparse, genMatrixesLungsHeart
import scipy.sparse as sp

from utils.dataset_remove import LandmarksDataset, ToTensor

In [3]:
test_heart = open("test_images_heart.txt",'r').read().splitlines()

test_jsrt = [image for image in test_heart if "JP" in image]
test_pad = [image for image in test_heart if not "JP" in image]

dataset_jsrt = LandmarksDataset(test_jsrt,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'LH',
                           transform = ToTensor()
                           )

dataset_pad = LandmarksDataset(test_pad,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'LH',
                           transform = ToTensor()
                           )

device = 'cuda:0'

In [4]:
A, AD, D, U = genMatrixesLungsHeart()

N1 = A.shape[0]
N2 = AD.shape[0]

A = sp.csc_matrix(A).tocoo()
AD = sp.csc_matrix(AD).tocoo()
D = sp.csc_matrix(D).tocoo()
U = sp.csc_matrix(U).tocoo()

D_ = [D.copy()]
U_ = [U.copy()]

config = {}

config['n_nodes'] = [N1, N1, N1, N2, N2, N2]
A_ = [A.copy(), A.copy(), A.copy(), AD.copy(), AD.copy(), AD.copy()]
A_t, D_t, U_t = ([scipy_to_torch_sparse(x).to(device) for x in X] for X in (A_, D_, U_))

config['latents'] = 64
config['inputsize'] = 1024

f = 32
config['filters'] = [2, f, f, f, f//2, f//2, f//2]
config['skip_features'] = f

hybrid_exp1 = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_exp1.load_state_dict(torch.load("weights/HybridGNet/Hybrid_exp_1/bestMSE.pt"))
hybrid_exp1.eval()

hybrid_exp2 = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_exp2.load_state_dict(torch.load("weights/HybridGNet/Hybrid_exp_2/bestMSE.pt"))
hybrid_exp2.eval()

hybrid_exp3 = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_exp3.load_state_dict(torch.load("weights/HybridGNet/Hybrid_exp_3/bestMSE.pt"))
hybrid_exp3.eval()

hybrid_exp4 = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_exp4.load_state_dict(torch.load("weights/HybridGNet/Hybrid_exp_4/bestMSE.pt"))
hybrid_exp4.eval()

print('Models loaded')

Models loaded


In [5]:
unet_exp1 = UNet(n_classes=3).to(device)
unet_exp1.load_state_dict(torch.load("weights/UNet/UNet_exp_1/bestDice.pt", map_location=device))
unet_exp1.eval()

unet_exp2 = UNet(n_classes=3).to(device)
unet_exp2.load_state_dict(torch.load("weights/UNet/UNet_exp_2/bestDice.pt", map_location=device))
unet_exp2.eval()

unet_exp3 = UNet(n_classes=3).to(device)
unet_exp3.load_state_dict(torch.load("weights/UNet/UNet_exp_3/bestDice.pt", map_location=device))
unet_exp3.eval()

unet_exp4 = UNet(n_classes=3).to(device)
unet_exp4.load_state_dict(torch.load("weights/UNet/UNet_exp_4/bestDice.pt", map_location=device))
unet_exp4.eval()

print('Models loaded')

Models loaded


In [6]:
UNet_HT_exp1 = UNet(n_classes=2).to(device)
UNet_HT_exp1.load_state_dict(torch.load("weights/UNet_HT/UNET_HT_EXP_1/bestDice.pt", map_location=device))
UNet_HT_exp1.eval()

UNet_HT_exp2 = UNet(n_classes=2).to(device)
UNet_HT_exp2.load_state_dict(torch.load("weights/UNet_HT/UNET_HT_EXP_2/bestDice.pt", map_location=device))
UNet_HT_exp2.eval()

UNet_HT_exp3 = UNet(n_classes=2).to(device)
UNet_HT_exp3.load_state_dict(torch.load("weights/UNet_HT/UNET_HT_EXP_3/bestDice.pt", map_location=device))
UNet_HT_exp3.eval()

UNet_HT_exp4 = UNet(n_classes=2).to(device)
UNet_HT_exp4.load_state_dict(torch.load("weights/UNet_HT/UNET_HT_EXP_4/bestDice.pt", map_location=device))
UNet_HT_exp4.eval()

print('Models loaded')

Models loaded


In [7]:
from medpy.metric import dc, hd
import cv2

def getDenseMask(RL, LL, H, imagesize = 1024):
    img = np.zeros([1024,1024])
    
    RL = RL.reshape(-1, 1, 2).astype('int')
    LL = LL.reshape(-1, 1, 2).astype('int')

    img = cv2.drawContours(img, [RL], -1, 1, -1)
    img = cv2.drawContours(img, [LL], -1, 1, -1)

    H = H.reshape(-1, 1, 2).astype('int')
    img = cv2.drawContours(img, [H], -1, 2, -1)
        
    return img

def evalImageMetrics(output, target):
    dcp = dc(output == 1, target == 1)
    dcc = dc(output == 2, target == 2)
    
    try:
        hdp = hd(output == 1, target == 1)
    except:
        hdp = -1

    try:
        hdc = hd(output == 2, target == 2)
    except:
        hdc = -1

    return [dcp, dcc, hdp, hdc]

import pandas as pd
from sklearn.metrics import mean_squared_error

hybrid_list = [hybrid_exp1, hybrid_exp2, hybrid_exp3,  hybrid_exp4]
hybrid_names = ['Hybrid exp 1', 'Hybrid exp 2', 'Hybrid exp 3', 'Hybrid exp 4']

unet_list = [unet_exp1, unet_exp2, unet_exp3, unet_exp4]
unet_names = ["UNet exp 1", "UNet exp 2", "UNet exp 3", "UNet exp 4"]

unet_ht_list = [UNet_HT_exp1, UNet_HT_exp2, UNet_HT_exp3, UNet_HT_exp4]
unet_ht_names = ["UNet HT exp 1", "UNet HT exp 2", "UNet HT exp 3", "UNet HT exp 4"]

datasets=[dataset_jsrt, dataset_pad]
dataset_names = ['JSRT', 'Padchest']

results = pd.DataFrame()

In [8]:
for k in range(0,2):
    dataset = datasets[k]

    for i in range(0, len(dataset.images)):
        print('\r',dataset_names[k], i+1,'of', len(dataset.images),end='')
        with torch.no_grad():
            sample = dataset[i]

            data, target = sample['image'], sample['landmarks']
            data = torch.unsqueeze(data, 0).to(device)
            target =  target.reshape(-1,2).numpy()
            
            RL = target[:44] * 1024
            LL = target[44:94] * 1024
            H = target[94:120] * 1024

            targetseg = getDenseMask(RL, LL, H)

            for j in range(0, len(hybrid_list)):
                output = hybrid_list[j](data)

                if len(output) > 1:
                    output = output[0]
                    
                output = output.cpu().numpy().reshape(-1, 2) 

                mse_l = mean_squared_error(output[:94,:].reshape(-1) * 1024, target[:94,:].reshape(-1) * 1024)
                mse_h = mean_squared_error(output[94:120,:].reshape(-1) * 1024, target[94:120,:].reshape(-1) * 1024)
                
                RL = output[:44] * 1024
                LL = output[44:94] * 1024
                H = output[94:120] * 1024

                outseg = getDenseMask(RL, LL, H)
                metrics = evalImageMetrics(outseg, targetseg)

                aux = pd.DataFrame([[i, j+1, dataset_names[k], hybrid_names[j], mse_l, mse_h] + metrics], 
                                     columns=['i', 'Exp', 'Dataset', 'Model', 'MSE Lungs', 'MSE Heart', 
                                     'Dice Lungs','Dice Heart','HD Lungs','HD Heart'])
                results = results.append(aux, ignore_index = True)

            for j in range(0, len(unet_list)):
                output = unet_list[j](data)
                outseg = torch.argmax(output[0,:,:,:], axis = 0).cpu().numpy()

                metrics = evalImageMetrics(outseg, targetseg)

                aux = pd.DataFrame([[i, j+1, dataset_names[k], unet_names[j], -1, -1] + metrics], 
                                     columns=['i', 'Exp', 'Dataset', 'Model', 'MSE Lungs', 'MSE Heart', 
                                     'Dice Lungs','Dice Heart','HD Lungs','HD Heart'])
                results = results.append(aux, ignore_index = True)

            
            for j in range(0, len(unet_ht_list)):
                output = unet_ht_list[j](data)
                sigmoid = torch.sigmoid(output)
                
                seg = torch.zeros([1024,1024])
                seg[sigmoid[0,0,:,:] > 0.5] = 1
                seg[sigmoid[0,1,:,:] > 0.5] = 2
                outseg = seg.cpu().numpy()
                
                metrics = evalImageMetrics(outseg, targetseg)

                aux = pd.DataFrame([[i, j+1, dataset_names[k], unet_ht_names[j], -1, -1] + metrics], 
                                     columns=['i', 'Exp', 'Dataset', 'Model', 'MSE Lungs', 'MSE Heart', 
                                     'Dice Lungs','Dice Heart','HD Lungs','HD Heart'])
                results = results.append(aux, ignore_index = True)
    print('')

 JSRT 49 of 49
 Padchest 27 of 27


In [9]:
for i in range(1,5):
    print('Experiment', i)

    for d in dataset_names:
        print(d)
        sub = results[results['Exp'] == i]
        sub = sub[sub['Dataset'] == d]
        del sub['i']
        sub = sub[['Dice Lungs', 'HD Lungs', 'Dice Heart', 'HD Heart', 'Model']]

        display(sub.groupby('Model').mean())

Experiment 1
JSRT


,Dice Lungs,HD Lungs,Dice Heart,HD Heart
Model,,,,
Hybrid exp 1,0.930768,47.236180,0.941262,31.190549
UNet HT exp 1,0.967673,90.264487,0.936967,80.022503
UNet exp 1,0.034056,82.530207,0.936890,56.468965


Padchest


,Dice Lungs,HD Lungs,Dice Heart,HD Heart
Model,,,,
Hybrid exp 1,0.951952,37.899251,0.942041,31.502155
UNet HT exp 1,0.960185,96.905789,0.928030,141.942535
UNet exp 1,0.955661,59.347753,0.934872,82.344530


Experiment 2
JSRT


...

,Dice Lungs,HD Lungs,Dice Heart,HD Heart
Model,,,,
Hybrid exp 3,0.904512,56.273676,0.945539,31.407110
UNet HT exp 3,0.932115,201.976400,0.930503,102.671946
UNet exp 3,0.034800,430.909379,0.921282,153.700651


Experiment 4
JSRT


,Dice Lungs,HD Lungs,Dice Heart,HD Heart
Model,,,,
Hybrid exp 4,0.974552,32.612979,0.939202,31.935399
UNet HT exp 4,0.979259,53.384362,0.942543,59.735841
UNet exp 4,0.979717,48.985193,0.944481,68.438881


Padchest


,Dice Lungs,HD Lungs,Dice Heart,HD Heart
Model,,,,
Hybrid exp 4,0.956863,38.118758,0.898703,69.655573
UNet HT exp 4,0.961036,63.451385,0.882060,188.501887
UNet exp 4,0.960725,95.043581,0.039269,72.131985
